# Notebook with final version of script providing data to backend

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
from datetime import datetime as dt
import re
import random

"""
generate_map_datapoints_df() - generuje DataFrame z wszystkimi datapointami na mapie

create_map_datapoints(df) - tworzy słownik z DataFrame'a 'df' z wszystkimi obiektami na mapie
Słownik jako key zawiera adresy, a jako value zawiera kolejny słownik z danymi,
np. '[Akacjowa 3': {'nr_zbiornika': 'A41312', 'st_oddanej_do_pobranej': 0.6265984654731442}, ...]


generate_quotient_timeseries_df() - generuje DataFrame z danymi z timeseries z deklarowanych ścieków
i pobieranej wody. Ten DataFrame jest potrzebny do dwóch funkcji poniżej


graph_quotient_timeseries(df, address) - generuje graf dla punktu o adresie 'address' z DataFrame'a 'df'
z stosunkiem wody oddanej do pobranej
Przykładowy graf ma formę: 
{
    'name': 'quotient_timeseries',
    'title': 'Stosunek wody zadeklarowanej jako ścieki do pobranej na przestrzeni miesięcy',
    'data': [
                {'date': '2021-11', 'quotient': 0.625}, 
                {'date': '2021-10', 'quotient': 0.66875}, 
                {'date': '2021-09', 'quotient': 0.6774193548387096}, 
                {'date': '2021-08', 'quotient': 0.7096774193548387}, 
                {'date': '2021-07', 'quotient': 0.7034482758620689}, 
                {'date': '2021-06', 'quotient': 0.7310344827586207}
            ]
}

graph_amount_timeseries(df, address) - generuje graf dla punktu o adresie 'address' z DataFrame'a 'df'
z iloscia wody pobranej i oddanej
Przykładowy graf ma formę: 
{
    'name': 'quotient_timeseries',
    'title': 'm^3 wody zadeklarowanej jako ścieki i pobranej na przestrzeni miesięcy',
    'data': [
                {'date': '2021-11', 'pobrana': 16.5, 'deklarowana': 10.0},
                {'date': '2021-10', 'pobrana': 16.5, 'deklarowana': 9.6},
                {'date': '2021-09', 'pobrana': 16.0, 'deklarowana': 9.6},
                {'date': '2021-08', 'pobrana': 16.0, 'deklarowana': 8.5},
                {'date': '2021-07', 'pobrana': 15.0, 'deklarowana': 11.1},
                {'date': '2021-06', 'pobrana': 15.0, 'deklarowana': 10.2}
            ]
}
"""


pd.options.mode.chained_assignment = None  

def generate_map_datapoints_df():
    waterConsumption_raw = pd.read_excel("data/waterConsumption.xlsx")
    
    # counting mean values to account for zeros
    
    for i in range(143):
        if waterConsumption_raw.iloc[i, 6] != 0:
            for j in range(17):
                temp = waterConsumption_raw.iloc[i, 6 + j * 2]
                waterConsumption_raw.iloc[i, 6 + j * 2] = temp / 2
                waterConsumption_raw.iloc[i, 6 + j * 2 + 1] = temp / 2
        else:
            for j in range(17):
                temp = waterConsumption_raw.iloc[i, 6 + j * 2 + 1]
                waterConsumption_raw.iloc[i, 6 + j * 2] = temp / 2
                waterConsumption_raw.iloc[i, 6 + j * 2 + 1] = temp / 2
        waterConsumption_raw.iloc[i, 40] = waterConsumption_raw.iloc[i, 39]
    
    # zmiana nazw kolumn
    waterConsumption = waterConsumption_raw.drop(['Lp.'], axis = 1)
    c = waterConsumption.columns
    waterConsumption = waterConsumption.rename(columns = {c[0]: 'nr_licznika',
                                                          c[1]: 'osoba',
                                                          c[2]: 'adres_licznika',
                                                          c[3]: 'zuzycie_wody',
                                                          c[4]: 'srednie_zuzucie_wody'})

    for i in range(5, len(c)-1):
        waterConsumption = waterConsumption.rename(columns = 
                                                   {c[i]: dt.strptime(str(c[i]), '%Y-%m').strftime('%Y-%m')})
    
    declaredSewage_raw = pd.read_excel("data/declaredSewage.xlsx")
    declaredSewage = declaredSewage_raw.drop(['Lp.'], axis = 1)
    c = declaredSewage.columns
    declaredSewage = declaredSewage.rename(columns = {c[0]: 'nr_zbiornika',
                                                      c[1]: 'adres_licznika',
                                                      c[2]: 'data_odbioru',
                                                      c[3]: 'deklaracja_mieszkaniec',
                                                      c[4]: 'deklaracja_firma',
                                                      c[5]: 'pobrana_woda',
                                                      c[6]: 'pobrana_woda_ogrodowa',
                                                      c[7]: 'nr_pojazdu'})
    
    # drop useless columns
    waterConsumption = waterConsumption.drop(columns = ['nr_licznika', 'osoba', 'zuzycie_wody'])
    waterConsumption= waterConsumption.iloc[: , :2]
    
    declaredSewage = declaredSewage.drop(columns = ['nr_zbiornika', 'data_odbioru', 'pobrana_woda', 
                                                        'pobrana_woda_ogrodowa', 'nr_pojazdu'])
    
    declaredSewage = declaredSewage.iloc[:, :3]
    declaredSewage['srednia_deklaracji'] = \
                   (declaredSewage['deklaracja_mieszkaniec'] + declaredSewage['deklaracja_firma'])/2
                   
    declaredSewage = declaredSewage.drop(columns = ['deklaracja_firma', 'deklaracja_mieszkaniec'])
    
   
    df = pd.DataFrame()
    
    df['adres'] = waterConsumption['adres_licznika']
    
    
    df['st_oddanej_do_pobranej'] = declaredSewage['srednia_deklaracji'] / waterConsumption['srednie_zuzucie_wody']
    
    
    return df

                                                    
def create_map_datapoints(df):
    data_dict = {}
    for index, row in df.iterrows():
        row_dict = {}

        if not np.isnan(row['st_oddanej_do_pobranej']): 
            row_dict['nr_zbiornika'] = random.choice("ABCDEFGHIJKL") + str(int(random.uniform(10000, 99999)//1))
            
            row_dict['st_oddanej_do_pobranej'] = row['st_oddanej_do_pobranej']        

        data_dict[row['adres']] = row_dict
    return data_dict



def generate_quotient_timeseries_df():
    waterConsumption_raw = pd.read_excel("data/waterConsumption.xlsx")
    
    # counting mean values to account for zeros
    
    for i in range(143):
        if waterConsumption_raw.iloc[i, 6] != 0:
            for j in range(17):
                temp = waterConsumption_raw.iloc[i, 6 + j * 2]
                waterConsumption_raw.iloc[i, 6 + j * 2] = temp / 2
                waterConsumption_raw.iloc[i, 6 + j * 2 + 1] = temp / 2
        else:
            for j in range(17):
                temp = waterConsumption_raw.iloc[i, 6 + j * 2 + 1]
                waterConsumption_raw.iloc[i, 6 + j * 2] = temp / 2
                waterConsumption_raw.iloc[i, 6 + j * 2 + 1] = temp / 2
        waterConsumption_raw.iloc[i, 40] = waterConsumption_raw.iloc[i, 39]
    
    # zmiana nazw kolumn
    waterConsumption = waterConsumption_raw.drop(['Lp.'], axis = 1)
    c = waterConsumption.columns
    waterConsumption = waterConsumption.rename(columns = {c[0]: 'nr_licznika',
                                                          c[1]: 'osoba',
                                                          c[2]: 'adres_licznika',
                                                          c[3]: 'zuzycie_wody',
                                                          c[4]: 'srednie_zuzucie_wody'})

    for i in range(5, len(c)-1):
        waterConsumption = waterConsumption.rename(columns = 
                                                   {c[i]: dt.strptime(str(c[i]), '%Y-%m').strftime('%Y-%m')})
    
    declaredSewage_raw = pd.read_excel("data/declaredSewage.xlsx")
    declaredSewage = declaredSewage_raw.drop(['Lp.'], axis = 1)
    c = declaredSewage.columns
    declaredSewage = declaredSewage.rename(columns = {c[0]: 'nr_zbiornika',
                                                      c[1]: 'adres_licznika',
                                                      c[2]: 'data_odbioru',
                                                      c[3]: 'deklaracja_mieszkaniec',
                                                      c[4]: 'deklaracja_firma',
                                                      c[5]: 'pobrana_woda',
                                                      c[6]: 'pobrana_woda_ogrodowa',
                                                      c[7]: 'nr_pojazdu'})
    
    # drop useless columns
    waterConsumption = waterConsumption.drop(columns = 
                                             ['nr_licznika', 'osoba', 'zuzycie_wody','srednie_zuzucie_wody'])
    
    declaredSewage = declaredSewage.drop(columns = ['nr_zbiornika', 'data_odbioru', 'deklaracja_mieszkaniec', 
                                       'deklaracja_firma', 'pobrana_woda', 'pobrana_woda_ogrodowa', 'nr_pojazdu'])
    
    waterConsumption= waterConsumption.iloc[: , :7]
    declaredSewage = declaredSewage.iloc[:, :7]
    
    df = pd.DataFrame()
    
    df['adres'] = waterConsumption['adres_licznika']
    
    for col_cons, col_decl in zip(waterConsumption.columns[1:], declaredSewage.columns[1:]):
        df[col_cons] = declaredSewage[col_decl] / waterConsumption[col_cons]
        df["pobrana_" + col_cons] =  waterConsumption[col_cons]
        df["deklarowana_" + col_cons] = declaredSewage[col_decl] 
        
    return df

def graph_quotient_timeseries(df, address):
    row = df[df['adres'] == address]
    
    graph = {}
    graph['name'] = 'quotient_timeseries'
    graph['title'] = "Stosunek zadeklarowanych ścieków do pobranej wody"
    
    data_list = []
    
    for col in row.columns[1:]:
        # if column is year and not pobrana or deklarowana
        if re.search("^20.*$", col):
            temp_dict = {}

            temp_dict['date'] = col
            temp_dict['quotient'] = float(row[col])

            data_list.append(temp_dict)
        
        
        
    graph['data'] = data_list
    
    return graph

def graph_amount_timeseries(df, address):
    row = df[df['adres'] == address]
    
    graph = {}
    graph['name'] = 'quotient_timeseries'
    graph['title'] = "Zadeklarowane ścieki i pobrana woda"
    
    data_list = []
    
    for col in row.columns[1:]:
        if re.search("^20.*$", col):
            temp_dict = {}
            temp_dict['date'] = col
            temp_dict['pobrana'] = float(row["pobrana_" + col])
            temp_dict['deklarowana'] = float(row["deklarowana_" + col])
            
            data_list.append(temp_dict)

        
    graph['data'] = data_list
    
    return graph

In [13]:
create_map_datapoints(generate_map_datapoints_df()) 

{'Roosevelta 164': {'nr_zbiornika': 'L47817',
  'st_oddanej_do_pobranej': 0.8139534883720899},
 'Paczkowskiego 6': {'nr_zbiornika': 'K75708',
  'st_oddanej_do_pobranej': 1.0309141215281472},
 'Kłeckoska 84': {'nr_zbiornika': 'C46223',
  'st_oddanej_do_pobranej': 0.649653178010538},
 'Zamiejska 13': {'nr_zbiornika': 'K14362',
  'st_oddanej_do_pobranej': 0.8710662425137756},
 'Kłeckoska 96 A': {'nr_zbiornika': 'C72434',
  'st_oddanej_do_pobranej': 0.5941417622244668},
 'Trzemeszeńska 2F': {'nr_zbiornika': 'I44433',
  'st_oddanej_do_pobranej': 0.3337352088686045},
 'Kłeckoska 51': {'nr_zbiornika': 'H76643',
  'st_oddanej_do_pobranej': 1.1091498524830703},
 'Roosevelta 131 A': {'nr_zbiornika': 'E98740',
  'st_oddanej_do_pobranej': 0.6999731517147307},
 'Skrajna 10': {'nr_zbiornika': 'E99258',
  'st_oddanej_do_pobranej': 0.7094594594594594},
 'Żerniki 6': {'nr_zbiornika': 'J73388',
  'st_oddanej_do_pobranej': 0.6666666666666667},
 'Skrajna 11': {'nr_zbiornika': 'D44925',
  'st_oddanej_do_po

In [14]:
generate_quotient_timeseries_df()

,adres,2021-11,pobrana_2021-11,deklarowana_2021-11,2021-10,pobrana_2021-10,deklarowana_2021-10,2021-09,pobrana_2021-09,deklarowana_2021-09,2021-08,pobrana_2021-08,deklarowana_2021-08,2021-07,pobrana_2021-07,deklarowana_2021-07,2021-06,pobrana_2021-06,deklarowana_2021-06
0,Roosevelta 164,0.818182,11.0000,9.0,0.827273,11.0000,9.1,0.866667,12.0000,10.4,0.791667,12.0000,9.5,0.900000,10.0000,9.0,0.800000,10.0000,8.0
1,Paczkowskiego 6,1.307692,6.5000,8.5,1.200000,6.5000,7.8,1.211735,6.2720,7.6,1.307398,6.2720,8.2,1.047120,9.0725,9.5,0.837696,9.0725,7.6
2,Kłeckoska 84,0.596178,16.7735,10.0,0.667720,16.7735,11.2,0.515050,21.1630,10.9,0.519775,21.1630,11.0,0.521184,17.0765,8.9,0.661728,17.0765,11.3
3,Zamiejska 13,0.961599,7.9035,7.6,0.873031,7.9035,6.9,0.915705,7.9720,7.3,0.878073,7.9720,7.0,0.733289,10.6370,7.8,0.658080,10.6370,7.0
4,Kłeckoska 96 A,0.567811,17.6115,10.0,0.516708,17.6115,9.1,0.219552,43.7255,9.6,0.196682,43.7255,8.6,0.685291,14.3005,9.8,0.615363,14.3005,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Wierzbiczany 71,0.571429,17.5000,10.0,0.554286,17.5000,9.7,0.544444,18.0000,9.8,0.600000,18.0000,10.8,0.554286,17.5000,9.7,0.497143,17.5000,8.7
139,Akacjowa 3,0.666667,10.5000,7.0,0.809524,10.5000,8.5,0.666667,10.5000,7.0,0.752381,10.5000,7.9,0.790000,10.0000,7.9,0.820000,10.0000,8.2
140,Myśliwiecka 25,0.677419,15.5000,10.5,0.754839,15.5000,11.7,0.580645,15.5000,9.0,0.761290,15.5000,11.8,0.726667,15.0000,10.9,0.613333,15.0000,9.2
141,Poznańśka 245,0.827586,14.5000,12.0,0.737931,14.5000,10.7,0.733333,15.0000,11.0,0.880000,15.0000,13.2,0.800000,14.5000,11.6,0.848276,14.5000,12.3


In [15]:
graph_quotient_timeseries(generate_quotient_timeseries_df(), "Grodzka 6")

{'name': 'quotient_timeseries',
 'title': 'Stosunek wody zadeklarowanej jako ścieki do pobranej na przestrzeni miesięcy',
 'data': [{'date': '2021-11', 'quotient': 0.6060606060606061},
  {'date': '2021-10', 'quotient': 0.5818181818181818},
  {'date': '2021-09', 'quotient': 0.6},
  {'date': '2021-08', 'quotient': 0.53125},
  {'date': '2021-07', 'quotient': 0.74},
  {'date': '2021-06', 'quotient': 0.6799999999999999}]}

In [16]:
graph_amount_timeseries(generate_quotient_timeseries_df(), "Grodzka 6")

{'name': 'quotient_timeseries',
 'title': 'm^3 wody zadeklarowanej jako ścieki i pobranej na przestrzeni miesięcy',
 'data': [{'date': '2021-11', 'pobrana': 16.5, 'deklarowana': 10.0},
  {'date': '2021-10', 'pobrana': 16.5, 'deklarowana': 9.6},
  {'date': '2021-09', 'pobrana': 16.0, 'deklarowana': 9.6},
  {'date': '2021-08', 'pobrana': 16.0, 'deklarowana': 8.5},
  {'date': '2021-07', 'pobrana': 15.0, 'deklarowana': 11.1},
  {'date': '2021-06', 'pobrana': 15.0, 'deklarowana': 10.2}]}